# Multi-Class Neural Network Implementation

This notebook implements a three hidden layer neural network for multi-class classification (5 classes) using **Sigmoid activation** for all layers.

## 1. Imports and Data Generation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import seaborn as sns
import pandas as pd

def generate_data(n_samples=2000, n_classes=5, n_features=2, random_state=42):
    """
    Generates a synthetic dataset.
    """
    X, y_raw = make_blobs(n_samples=n_samples, centers=n_classes, n_features=n_features, random_state=random_state, cluster_std=1.5)
    
    # One-hot encode
    encoder = OneHotEncoder(sparse_output=False)
    y = encoder.fit_transform(y_raw.reshape(-1, 1))
    
    return X, y, y_raw

# Generate Data
print("Generating data...")
X, y, y_raw = generate_data()

# Save to CSV
df = pd.DataFrame(X, columns=[f'feature_{i+1}' for i in range(X.shape[1])])
df['label'] = y_raw
df.to_csv('synthetic_data.csv', index=False)
print("Data saved to synthetic_data.csv")

# Load from CSV
print("Loading data from synthetic_data.csv...")
df_loaded = pd.read_csv('synthetic_data.csv')
X = df_loaded.drop('label', axis=1).values
y_raw = df_loaded['label'].values

# Re-encode labels (since we loaded raw labels)
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y_raw.reshape(-1, 1))
print("Data loaded and re-encoded.")

# Standardize Data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 2. Neural Network Class
Implementing the Neural Network with 3 hidden layers and Sigmoid activation.

In [ ]:
class NeuralNetwork(object):
    def __init__(self, input_size=2, hidden_size=10, output_size=5, learning_rate=0.1):
        """
        Initializes the Neural Network with 3 hidden layers.
        """
        self.inputLayerNeurons = input_size
        self.hiddenLayerNeurons = hidden_size
        self.outLayerNeurons = output_size
        self.learning_rate = learning_rate

        # Initialize weights
        # Layer 1: Input -> Hidden 1
        self.W1 = np.random.randn(self.inputLayerNeurons, self.hiddenLayerNeurons)
        # Layer 2: Hidden 1 -> Hidden 2
        self.W2 = np.random.randn(self.hiddenLayerNeurons, self.hiddenLayerNeurons)
        # Layer 3: Hidden 2 -> Hidden 3
        self.W3 = np.random.randn(self.hiddenLayerNeurons, self.hiddenLayerNeurons)
        # Layer 4: Hidden 3 -> Output
        self.W4 = np.random.randn(self.hiddenLayerNeurons, self.outLayerNeurons)

    def sigmoid(self, x, der=False):
        if der == True:
            return x * (1 - x)
        else:
            return 1 / (1 + np.exp(-x))

    def feedForward(self, X):
        # Hidden Layer 1
        self.z1 = np.dot(X, self.W1)
        self.a1 = self.sigmoid(self.z1)

        # Hidden Layer 2
        self.z2 = np.dot(self.a1, self.W2)
        self.a2 = self.sigmoid(self.z2)

        # Hidden Layer 3
        self.z3 = np.dot(self.a2, self.W3)
        self.a3 = self.sigmoid(self.z3)

        # Output Layer
        self.z4 = np.dot(self.a3, self.W4)
        self.pred = self.sigmoid(self.z4)

        return self.pred

    def backPropagation(self, X, Y, pred):
        # Output Layer Error
        output_error = Y - pred
        delta4 = self.learning_rate * output_error * self.sigmoid(pred, der=True)

        # Hidden Layer 3 Error
        error3 = np.dot(delta4, self.W4.T)
        delta3 = self.learning_rate * error3 * self.sigmoid(self.a3, der=True)

        # Hidden Layer 2 Error
        error2 = np.dot(delta3, self.W3.T)
        delta2 = self.learning_rate * error2 * self.sigmoid(self.a2, der=True)

        # Hidden Layer 1 Error
        error1 = np.dot(delta2, self.W2.T)
        delta1 = self.learning_rate * error1 * self.sigmoid(self.a1, der=True)

        # Update Weights
        self.W4 += np.dot(self.a3.T, delta4)
        self.W3 += np.dot(self.a2.T, delta3)
        self.W2 += np.dot(self.a1.T, delta2)
        self.W1 += np.dot(X.T, delta1)

    def train(self, X, Y):
        output = self.feedForward(X)
        self.backPropagation(X, Y, output)
        return output

    def predict(self, X):
        probs = self.feedForward(X)
        return np.argmax(probs, axis=1)

## 3. Helper Functions

In [ ]:
def plot_loss(loss_history):
    plt.figure(figsize=(10, 6))
    plt.plot(loss_history)
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.grid(True)
    plt.show()

def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

## 4. Training and Evaluation

In [ ]:
# Initialize Neural Network
input_size = X.shape[1]
output_size = y.shape[1]
nn = NeuralNetwork(input_size=input_size, hidden_size=10, output_size=output_size, learning_rate=0.1)

# Train
print("Training model...")
epochs = 5000
loss_history = []

for i in range(epochs):
    output = nn.train(X_train, y_train)
    
    # Calculate MSE Loss for monitoring (since we are using Sigmoid output)
    loss = np.mean(np.square(y_train - output))
    loss_history.append(loss)
    
    if i % 500 == 0:
        print(f"Epoch {i}, Loss: {loss:.4f}")
        
print(f"Final Loss: {loss:.4f}")
plot_loss(loss_history)

# Evaluate
print("\nEvaluating model...")
y_pred_probs = nn.feedForward(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_test, axis=1)

# Metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Confusion Matrix
plot_confusion_matrix(y_true, y_pred, classes=[0, 1, 2, 3, 4])